In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install plotly

In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [17]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [18]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [19]:
amazon_df

,title,price,rating,reviews,availability
0,Sony Playstation PS4 1TB Black Console,,4.7 out of 5 stars,"1,056 ratings",In Stock
1,PlayStation 4 500GB Console (Renewed),,4.0 out of 5 stars,443 ratings,In Stock.
2,PlayStation 4 Slim 1TB Console - Black (Renewed),,4.4 out of 5 stars,"1,226 ratings",In Stock.
3,DualShock 4 Wireless Controller for PlayStatio...,,4.7 out of 5 stars,"145,974 ratings",In Stock
4,Logitech G29 Driving Force Racing Wheel and Fl...,,4.7 out of 5 stars,"16,329 ratings",In Stock
5,Sonic Frontiers - PlayStation 4,,4.7 out of 5 stars,493 ratings,In Stock
6,BlueFire Stereo Gaming Headset for Playstation...,,4.4 out of 5 stars,"8,442 ratings",In Stock
7,PS4 Fan Cooling Fan with Controller Charger Co...,,4.3 out of 5 stars,"1,349 ratings",In Stock
8,Sony PlayStation 4 Slim Limited Edition 1TB Ga...,,4.3 out of 5 stars,"1,058 ratings",In Stock.
9,RPlay Play-Station 4 PS4 1TB Slim Edition Jet ...,,4.8 out of 5 stars,8 ratings,Only 5 left in stock - order soon
